In [4]:
pip install paho-mqtt

     ---------------------------------------- 0.0/99.4 kB ? eta -:--:--
     ---------------------------------------- 99.4/99.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=65646 sha256=372ea6f391752de20e5bddd5f5f4029fbc3e232907d065af88f3b70a552fb691
  Stored in directory: c:\users\lee\appdata\local\pip\cache\wheels\29\ea\a5\ba9a63aaf4cd4e16e8a87ee31fb4d11b04ff5e1735d312619a
Successfully built paho-mqtt
Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB 2.3 MB/s eta 0:00:00


In [27]:
pip install --upgrade certifi

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import paho.mqtt.client as mqtt
import pymysql
import json
from datetime import datetime, timedelta
import time
import threading
import numpy as np
import joblib

# 쓰레드를 정지시킬 이벤트
stop_event = threading.Event()

# 머신러닝 모델 불러오기
model_filename = "posture_model.pkl"
loaded_model = joblib.load(model_filename)

# MySQL 연결 설정
connection = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='ttap',
    password='1234',
    database='ttap',
    cursorclass=pymysql.cursors.DictCursor
)


cplevel = None  
dplevel = None  

# 사용자 설정 높이 조회
def check_height():
    global cplevel, dplevel
    try:
        with connection.cursor() as cursor:
            sql = "SELECT cplevel, dplevel FROM user_settings WHERE member_id = 'test1'"
            cursor.execute(sql)
            result = cursor.fetchone()

            if result:
                cplevel = result['cplevel']
                dplevel = result['dplevel']
                print(cplevel)
                print(dplevel)
    except pymysql.Error as e:
        print(f"Error checking height: {e}")
# 사용자 설정 높이 변경 

def change_cplevel(payload):
    global cplevel
    try:
        new_cplevel = json.loads(payload)['new_cplevel']
        # 여기에서 필요한 유효성 검사 등을 수행

        # 데이터베이스 업데이트
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET cplevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (new_cplevel,))
            connection.commit()

        cplevel = new_cplevel
        print(f"CP Level updated: {cplevel}")
        check_height()
        # Publish updated height information to "Apill/height" topic
        client.publish("Apill/height", json.dumps({"cplevel": cplevel, "dplevel": dplevel}))

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing CP level: {e}")

def change_dplevel(payload):
    global dplevel
    try:
        new_dplevel = json.loads(payload)['new_dplevel']
        # 여기에서 필요한 유효성 검사 등을 수행

        # 데이터베이스 업데이트
        with connection.cursor() as cursor:
            sql = "UPDATE user_settings SET dplevel = %s WHERE member_id = 'test1'"
            cursor.execute(sql, (new_dplevel,))
            connection.commit()

        dplevel = new_dplevel
        print(f"DP Level updated: {dplevel}")
        check_height()
        # Publish updated height information to "Apill/height" topic
        client.publish("Apill/height", json.dumps({"cplevel": cplevel, "dplevel": dplevel}))

    except (json.JSONDecodeError, KeyError, pymysql.Error) as e:
        print(f"Error changing DP level: {e}")


def process_sensor_data(payload):
    try:
        # 문자열을 쉼표로 분할하고 양쪽 공백 제거
        sensor_values = [value.strip() for value in payload.replace('"', '').split(',') if value.strip()]

        if len(sensor_values) == 26:
            # 모든 값을 실수로 변환
            sensor_values = list(map(float, sensor_values))

            # NumPy 배열로 변환하고 reshape
            input_data = np.array(sensor_values).reshape(1, -1)

            # 예측
            predicted_label = loaded_model.predict(input_data)[0]

            # 현재 시간 가져오기
            current_time = datetime.now().strftime("%H:%M")

            with connection.cursor() as cursor:
                # 정보를 데이터베이스에 추가
                sql = "INSERT INTO sleepposture (posture, time) VALUES (%s, %s)"
                cursor.execute(sql, (predicted_label, current_time))
                connection.commit()

            # 분류 결과 확인
            if predicted_label[-2:] == 'DP':
                # 예측 결과가 'DP'로 끝날 경우 DPLEVEL 전송
                client.publish("Apill/height", json.dumps({"level": dplevel}))
            elif predicted_label[-2:] == 'CP':
                # 예측 결과가 'CP'로 끝날 경우 CPLEVEL 전송
                client.publish("Apill/height", json.dumps({"level": cplevel}))

            # 예측 결과를 MQTT 메시지로 전송 (실수로 변환하여 전송)
#             client.publish("Apill/sleepposture", str(predicted_label))

            print(f"Posture recorded - Sensor Values: {sensor_values}, Predicted Label: {predicted_label}, Time: {current_time}")

    except Exception as e:
        print(f"Error processing sensor data: {e}")
# 알람 목록을 저장할 리스트
alarms = []

# 알람 목록 조회 함수
def get_alarms():
    global alarms
    try:
        with connection.cursor() as cursor:
            # 알람 목록을 데이터베이스에서 조회
            sql = "SELECT id, time, isOn, isSelected FROM alarms"
            cursor.execute(sql)
            alarms = cursor.fetchall()

        # datetime 객체를 문자열로 변환 또는 None인 경우 그대로 둠
        for alarm in alarms:
            if isinstance(alarm['time'], datetime):
                alarm['time'] = alarm['time'].strftime("%H:%M")

        # 알람 목록을 MQTT로 전송
#         client.publish("Apill/alarm/return", json.dumps(alarms))
    except pymysql.Error as e:
        print(f"Error getting alarms: {e}")

# 주기적으로 현재 시간을 확인하고 알람이 발생해야 하는지 여부를 판단하는 함수
def check_and_trigger_alarm_periodically():
    while not stop_event.is_set():
        try:
            get_alarms()
            current_time = datetime.now().replace(microsecond=0).time()

            for alarm in alarms:
                alarm_time = datetime.strptime(alarm['time'], "%H:%M").time()
                if alarm_time == current_time and alarm['isOn']:
                    print(f"Publishing triggered message to Apill/alarm/return")
                    client.publish("Apill/alarm/return", "triggered")

            time.sleep(60)  # 1초 대신 60초로 수정
        except Exception as e:
            print(f"Error in check_and_trigger_alarm_periodically: {e}")
            
# 알람 추가 함수
def add_alarm(payload):
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에 추가
            sql = "INSERT INTO alarms (time, isOn, isSelected) VALUES (%s, %s, %s)"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected']))
            connection.commit()
        print(f"Alarm added: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error adding alarm: {e}")

# 알람 수정 함수
def update_alarm(payload):
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 수정
            sql = "UPDATE alarms SET time=%s, isOn=%s, isSelected=%s WHERE id=%s"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected'], alarm_data['id']))
            connection.commit()
        print(f"Alarm updated: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error updating alarm: {e}")

# 알람 삭제 함수
def delete_alarm(payload):
    try:
        alarm_id = json.loads(payload)['id']
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 삭제
            sql = "DELETE FROM alarms WHERE id=%s"
            cursor.execute(sql, (alarm_id,))
            connection.commit()
        print(f"Alarm deleted: {alarm_id}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error deleting alarm: {e}")        
        
# SQL 문을 실행하고 결과를 MQTT로 전송하는 함수
def execute_sql_query(sql_query):
    try:
        print(sql_query)
        with connection.cursor() as cursor:
            # SQL 실행
            cursor.execute(sql_query)
            # 결과 가져오기
            result = cursor.fetchall()
            # 결과를 MQTT 토픽에 전송
            json_result = json.dumps(result, default=str)
            print(f"SQL Query Result: {json_result}")
            client.publish("Apill/sql/return", json_result)
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/sql/return", json.dumps(error_message, default=str))
        
        
# JOIN SQL 문을 실행하고 결과를 MQTT로 전송하는 함수
def insert_sql_join(msg):
    try:
        print(msg)
        
        # msg를 쉼표로 분리하여 리스트로 만들기
        msg_list = [value.strip() for value in msg.split(",")]
        
        with connection.cursor() as cursor:
            # SQL 실행
            sql = '''
                    INSERT INTO members (
                        member_id, member_pw, member_name, member_birth, 
                        member_weight, member_height, member_gender, member_age
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s
                    )
                    '''
            cursor.execute(sql, msg_list)
            # 영향 받은 행의 수 확인
            affected_rows = cursor.rowcount
            # 결과를 MQTT 토픽에 전송
            json_result = json.dumps(affected_rows, default=str)
            print(f"SQL Query Result: {json_result}")
            client.publish("Apill/sql/join/return", json_result)
    except pymysql.Error as e:
        error_message = f"Error executing SQL: {e}"
        print(error_message)
        client.publish("Apill/sql/join/return", json.dumps(error_message, default=str))        


# MQTT 브로커 설정
broker_address = "172.30.1.21"
broker_port = 1883

# MQTT 클라이언트 생성
client = mqtt.Client()

# MQTT 연결 시 호출되는 콜백 함수
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    
    # 토픽 구독
    # 센서값
    client.subscribe("Apill/RB/sensor")
#     client.subscribe("Apill/sleepposture")

    # 알람
    client.subscribe("Apill/alarm/add")
    client.subscribe("Apill/alarm/update")
    client.subscribe("Apill/alarm/delete")
    client.subscribe("Apill/alarm/get")
    
    # sql문 처리
    client.subscribe("Apill/sql")

# MQTT 메시지 수신 시 호출되는 콜백 함수
def on_message(client, userdata, msg):
    print(f"Received message: {msg.payload.decode()}")

    # 센서값 처리
    if msg.topic == "Apill/RB/sensor":
        process_sensor_data(msg.payload.decode())
    elif msg.topic == "apill/height/cpchange":
        change_cplevel()
    elif msg.topic == "apill/height/dpchange":
        change_dplevel()
    # 알람 목록 조회 메시지 처리
    elif msg.topic == "Apill/alarm/get":
        get_alarms()
    # 알람 추가 메시지 처리
    elif msg.topic == "Apill/alarm/add":
        add_alarm(msg.payload.decode())
    # 알람 수정 메시지 처리
    elif msg.topic == "Apill/alarm/update":
        update_alarm(msg.payload.decode())
    # 알람 삭제 메시지 처리
    elif msg.topic == "Apill/alarm/delete":
        delete_alarm(msg.payload.decode())
        
    # SQL SELECT 문을 받아서 실행하고 결과를 MQTT로 전송
    elif msg.topic == "Apill/sql":
        execute_sql_query(msg.payload.decode())
    # SQL JOIN 문을 받아서 실행하고 결과를 MQTT로 전송
    elif msg.topic == 'Apill/sql/join':
        insert_sql_join(msg.payload.decode(utf-8))

# MQTT 클라이언트에 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

# MQTT 브로커에 연결
client.connect(broker_address, broker_port, 60)
check_height()
# 시작
# 알람을 주기적으로 확인하는 쓰레드 시작
alarm_thread = threading.Thread(target=check_and_trigger_alarm_periodically)
alarm_thread.start()

# 브로커 서버 루프 시작
try:
    client.loop_forever()
except KeyboardInterrupt:
    print("KeyboardInterrupt: Stopping threads and disconnecting MQTT.")
    stop_event.set()  # 쓰레드 종료를 위해 이벤트 설정
    alarm_thread.join()  # 쓰레드 종료 대기
    client.disconnect()  # MQTT 연결 종료
except Exception as e:
    print(f"Unhandled exception: {e}")

print("Exiting program.")

Connected with result code 0
Received message: SELECT sleep_num FROM ttap.mibanddata
